In [ ]:
#necessary imports

import os

import pandas as pd

import duckdb as db

In [ ]:
#handles large data visualization
def data_vis(query):
    if len(query) > 20:
        query = query.fetchall()
        for tpl in query:
            for element in tpl:
                print(element)
    else:
        return query

In [ ]:
#sets the paths to .csv files
path_netflix = os.path.abspath('../data/netflix_titles.csv')

path_amazon_prime = os.path.abspath('../data/amazon_prime_titles.csv')

data_amazon = pd.read_csv(path_amazon_prime)

data_netflix = pd.read_csv(path_netflix)

In [ ]:
#Creates temp table to merge the two pandas dataframes, data_netflix and data_amazon. A new column was added
#to distinguish the streaming platform
db.sql(
    """
    CREATE TEMP TABLE IF NOT EXISTS streaming_platforms AS
    
    WITH sub_amazon_netflix AS(
        SELECT
            *,
            'amazon_prime' AS platform
        
        FROM data_amazon

        UNION ALL

        SELECT
            *,
            'netflix' AS platform
        
        FROM data_netflix
    )
    SELECT *
    
    FROM sub_amazon_netflix
    """
    )

In [ ]:
# 1- Top 10 atores/atrizes considerando todos os dados
#point of attention: null titles, null cast
#There are elements in cast that, maybe, aren't names, but wrong data. Query ignoring one length elements

query_1 = db.query(
    """
    WITH sub_actor_actress AS (
        SELECT
            DISTINCT(title),
            --cast stored as a string, each name separeted with comma
            TRIM(UNNEST(STR_SPLIT(streaming_platforms.cast, ','))) AS actor_actress,
                
        FROM streaming_platforms

        WHERE streaming_platforms.cast IS NOT NULL
            AND title IS NOT NULL
    )

    SELECT
        actor_actress,
        COUNT(actor_actress) AS frequency_actor_actress,
    
    FROM sub_actor_actress
    --ignoring one length elements
    WHERE LEN(actor_actress) != 1

    GROUP BY actor_actress

    ORDER BY frequency_actor_actress DESC

    LIMIT 10
    """
)

data_vis(query_1)

In [ ]:
# 2- Top 5 países produtores de conteúdos considerando todos os dados e comparando as duas plataformas;
#points of attention: both streaming platforms

query_2 = db.query(
    """
        SELECT
            country,
            COUNT(DISTINCT title) AS country_production
        
        FROM streaming_platforms

        WHERE country IS NOT NULL

        GROUP BY country

        ORDER BY country_production DESC

        LIMIT 5
    """
)
data_vis(query_2)

In [ ]:
# 3- Mês no qual há mais adições de filmes na plataforma Netflix;
#points of attention: filter for movie, by month, on netflix platform

query_3 = db.query(
    """
        SELECT
        --date in string format. Necessary to convert in date data format
            DATE_TRUNC('MONTH', STRPTIME(date_added, '%B %d, %Y')) AS month_date_added,
            COUNT(DISTINCT title) AS total_movies_released

        FROM streaming_platforms

        WHERE platform = 'netflix'
            AND type = 'Movie'

        GROUP BY month_date_added

        ORDER BY total_movies_released DESC

        LIMIT 1
    """
)
data_vis(query_3)

In [ ]:
#4- Quantidade de filmes listados como comédia.
# points of attention: filter for movie type and comedy genre, either, each and both streaming platforms

query_4 = db.query(
    """
        SELECT
            platform,
            COUNT(DISTINCT title) AS total_comedy_movies

        FROM streaming_platforms

        WHERE type = 'Movie'
            AND listed_in IN ('Comedies', 'Comedy')
        
        GROUP BY platform

    """

)
data_vis(query_4)

In [ ]:
#5- Lista de todos os gêneros de filmes.
#points of attention: distinct genre, only movies

query_5 = db.sql(
    """
        SELECT
            DISTINCT(TRIM(UNNEST(STR_SPLIT(listed_in, ','))))

        FROM streaming_platforms

        WHERE type = 'Movie'
        
    """
)
data_vis(query_5)

In [ ]:
# 6- A frequência de "TV Show" de todos os dados e comparativamente em relação as duas plataformas
#points of attention: same TV Shows on both platforms must be counted once. Only TV Show
query_6 = db.query(
    """
        SELECT
            SUM(CASE WHEN platform = 'netflix' THEN 1 ELSE 0 END) AS tv_shows_on_netflix,
            
            SUM(CASE WHEN platform = 'amazon_prime' THEN 1 ELSE 0 END) AS tv_shows_on_amazon_prime,
            
            COUNT(DISTINCT title) AS total_tv_show_on_both_platforms
        
        FROM streaming_platforms

        WHERE type = 'TV Show'
                
    """
)
data_vis(query_6)

In [ ]:
#7- A frequência de "Movies" de todos os dados e comparativamente em relação as duas plataformas

query_7 = db.query(
    """
        SELECT
            SUM(CASE WHEN platform = 'netflix' THEN 1 ELSE 0 END) AS tv_shows_on_netflix,
            
            SUM(CASE WHEN platform = 'amazon_prime' THEN 1 ELSE 0 END) AS tv_shows_on_amazon_prime,
            
            COUNT(DISTINCT title) AS total_movies_on_both_platforms
        
        FROM streaming_platforms

        WHERE type = 'Movie'
                
    """
)
data_vis(query_7)